In [ ]:
#coding:utf-8
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import keras_metrics as km
%matplotlib
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[                     
                     'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)',#13
    
                      'Fe Precursor Ratio',#14
                      'Zn Precursor Ratio',#15
                      '2-Methylimidazole Precursor Ratio',#16
                      'Additive (Other) Precursor Ratio',#17
                      'ZIF Preparation Temperature (℃)',#18
                      'ZIF Preparation Time (h)',#19
                      'One-Pot Method',#20
                      'Post-Adsorb Method',#21
                      'Carbon-Derivative Adsorb Method',#22
                      'Pyrolysis Temperature (℃)',#23
                      'Pyrolysis Time (h)',#24
                      'Temperature Rising Rate (℃ min-1)',#25
                      'Seconde Pyrolysis',#26
                      'Acid Wash'#27
                        ]]
chemical_properies=raw_data.iloc[:,0:15]
synthesis=raw_data.iloc[:,15:29]
CL_78=fdata.loc[:,[                     
                      'HW_0.78'#27
]]
CL_80=fdata.loc[:,[                     
                      'HW_0.8'#27
]]
CL_82=fdata.loc[:,[                     
                      'HW_0.82'#27
]]
CL_003=fdata.loc[:,[                     
                      'PYMT_0.03'#27
]]
CL_002=fdata.loc[:,[                     
                      'PYMT_0.02'#27
]]
REG_HWP=fdata.loc[:,[                     
                      'Half Wave Potential in Acid Condition (vs. RHE)'#27
]]
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total
from sklearn.decomposition import PCA
pca_chem_3 = PCA(n_components=3)
chem_3 = pca_chem_3.fit_transform(chemical_properies)     

pca_synthesis_3 = PCA(n_components=3)
synthesis_3 = pca_synthesis_3.fit_transform(synthesis)  

pca_chem_2 = PCA(n_components=2)
chem_2 = pca_chem_2.fit_transform(chemical_properies)     

pca_synthesis_2 = PCA(n_components=2)
synthesis_2 = pca_synthesis_2.fit_transform(synthesis)    

print('ready')


In [ ]:
print('ready')
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import lightgbm
import catboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
from fancyimpute import KNN
from keras.callbacks import EarlyStopping 

early_stopping=keras.callbacks.EarlyStopping(
 monitor="val_loss", 
 patience=20, 
 verbose=0, 
 mode="auto"
)
###########fix random seed for reproducability##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=sum(absError)/len(absError)
    return mae,mse,RMSE


X=synthesis_3
y=CL_002.values.astype(np.float32)

###########fix random seed for reproducability##########
seed=406
###########train test splitting##########
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15,random_state=seed)
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})
def intergate(y_pred):
    length=y_pred.shape[0]
    print(length)
    for i in range (0,length):
        if y_pred[i][0]>=0.5:
            y_pred[i][0]=1
        else:
            y_pred[i][0]=0
    return y_pred
accuracy={}
for neurons1 in [50,100,200]:
    for dropout_rate in [0,0.25,0.5]:
        for batch_size_number in [16,32]:
            for reg in [0,0.0001,0.001]:
                for act in ['sigmoid','tanh','relu','softsign']:                        
                    for epochs_number in [200,400,800]:
                        regularizer=keras.regularizers.l2(reg)
                        ###########keras ANN model construction########## 
                        model = Sequential() 
                        model.add(Dense(neurons1, input_dim=3, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation='tanh',kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))                        
                        model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=act,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))
                        model.add(Dense(1, input_dim=neurons1, activation='sigmoid'))
                        model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=[km.f1_score(), km.binary_precision(), km.binary_recall()])
                        print('training...')
                        model.fit(X_train, y_train,verbose=0, batch_size=batch_size_number,epochs=epochs_number,validation_split=0.2,callbacks=[early_stopping])

                        print(neurons1,epochs_number,dropout_rate,batch_size_number,reg,act)
                        y_pred_test=model.predict(X_test)
                        y_pred_train=model.predict(X_train)
                        y_pred_test=intergate(y_pred_test)
                        y_pred_train=intergate(y_pred_train)
                        print(classification_report(y_train,y_pred_train))
                        print(classification_report(y_test,y_pred_test))
                        final_result=classification_report(y_test,y_pred_test,output_dict=True)
                        ac=final_result['accuracy']
                        accuracy[ac]=[neurons1,epochs_number,dropout_rate,batch_size_number,reg,act]
                        K.clear_session()    

In [ ]:
print(accuracy)